### 1. Import modules

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import re
import pickle
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import RobustScaler

In [2]:
# True on 'instances' means sample 10% of the data, to speed up the script
TEST_instances = False

# True on 'iterations' means run the script for (only) 3 iterations instead of 10
TEST_iterations = False

### 2. Merge .pkl's

In [3]:
# schoolgaanden
schoolgaanden = pd.read_pickle('4_Data\Pickles\schoolgaanden.pkl')

In [4]:
# Replace all types of whitespace characters with an underscore and delete all special characters
schoolgaanden.columns = schoolgaanden.columns.str.replace(r'\s+', '_', regex=True)
schoolgaanden.columns = schoolgaanden.columns.str.replace(r'\W+', '', regex=True)

In [ ]:
for column in schoolgaanden.columns:
    print(f'\'{column}\',')

In [6]:
# # Loop over every column name in a df
# def column_cleaner(df):
#     for column in df.columns:
#         ## Remove categories
#         # Get position of last capital letter by iterating backwards
#         pos = 0
#         for i in range(len(column)-1, 0, -1):
#             if column[i].isupper() and (column[i-1].islower() or column[i-1] == '_'):
#                 pos = i
#                 break
#         # If there is a capital letter
#         if pos != 0 and column[-1] != '_':
#             # Remove everything after the before capital letter
#             df.rename(columns={column: column[pos:]}, inplace=True)
#         elif pos != 0 and column[-1] == '_':
#             df.rename(columns={column: column[pos:] + '%'}, inplace=True)
#     return df

# # Manually set some other column names
# def column_editor(df):
#     df.rename(columns={'Inkomen_van_huishoudens40_huishoudens_met_laagste_inkomen_%': 'Huishoudens_laagste_landelijke_40%_huishoudensinkomens_%'}, inplace=True)
#     df.rename(columns={'Inkomen_van_huishoudens20_huishoudens_met_hoogste_inkomen_%': 'Huishoudens_hoogste_landelijke_20%_huishoudensinkomens_%'}, inplace=True)
#     df.rename(columns={'Huishoudens_tot_110_van_sociaal_minimum_%': 'Huishoudens_tot_110%_van_sociaal_minimum_%'}, inplace=True)
#     df.rename(columns={'Huishoudens_tot_120_van_sociaal_minimum_%': 'Huishoudens_tot_120%_van_sociaal_minimum_%'}, inplace=True)
#     df.rename(columns={'Personenautos_per_huishouden_per_huishouden': 'Personenautos_per_huishouden'}, inplace=True)
#     df.rename(columns={'stedelijkheid_1.0': 'zeer_sterk_stedelijk'}, inplace=True)
#     df.rename(columns={'stedelijkheid_2.0': 'sterk_stedelijk'}, inplace=True)
#     df.rename(columns={'stedelijkheid_3.0': 'matig_stedelijk'}, inplace=True)
#     df.rename(columns={'stedelijkheid_4.0': 'weinig_stedelijk'}, inplace=True)
#     df.rename(columns={'stedelijkheid_5.0': 'niet_stedelijk'}, inplace=True)
#     return df

# overzicht = schoolgaanden.copy()
# overzicht = column_cleaner(overzicht)
# overzicht = column_editor(overzicht)
# overzicht = pd.DataFrame(overzicht.isna().mean().round(2), columns=['Proporties ontbrekende waarden'])
# overzicht.to_csv('schoolgaanden_overzicht.csv')

### 3. Engineer features

#### 3.1 Continuous numerical

##### 3.1.1 Direct features

In [7]:
# Get columns that need no further transformation
X_schoolgaanden_numericals = schoolgaanden[[
    # 'Postcode',                               # Useless
    # 'Huisnummer',                             # Useless
    # 'Clientnummer',                           # Useless 
    'BMR',                                      # Drop after missing value imputation
    # 'DKTP',                                   # Related to target
    # 'DKTPbooster',                            # Related to target
    'DTP',                                      # Drop after missing value imputation
    # 'Hib',                                    # Related to target
    # 'Pneu',                                   # Related to target
    # 'Excluded',                               # Related to target
    # 'Hepatitis_B',                            # Related to target
    # 'MenACWYMenC',                            # Related to target
    # 'Geboortejaarmaand',                      # We construct age from this
    # 'Tweeling',                               # Boolean
    # 'Buurt_BRP',                              # Useless
    # 'CBS_Buurtcode_BRP',                      # Useless
    # 'GeboortejaarmaandBiologischeMoeder',     # We construct age from this
    # 'LeeftijdBiologischeMoederBijGeboorte',   # We construct age from this
    # 'Geslacht',                               # Boolean
    # 'Geboorteland',                           # Categorical
    # 'Schoolgaanden_status_BMR',               # Related to target
    # 'Schoolgaanden_status_DTP',               # Related to target
    # 'Schoolgaanden_status',                   # Related to target
    # 'Toestemming',                            # Boolean
    'aantal_inwoners',
    'aantal_mannen',
    'aantal_vrouwen',
    'aantal_inwoners_0_tot_15_jaar',
    'aantal_inwoners_15_tot_25_jaar',
    'aantal_inwoners_25_tot_45_jaar',
    'aantal_inwoners_45_tot_65_jaar',
    # 'aantal_inwoners_65_jaar_en_ouder',       # nth column in category
    'percentage_geb_nederland_herkomst_nederland',            # Ordinal, but many categories
    # 'percentage_geb_nederland_herkomst_overig_europa',        # Ordinal
    # 'percentage_geb_nederland_herkomst_buiten_europa',        # Ordinal
    # 'percentage_geb_buiten_nederland_herkomst_europa',        # Ordinal
    # 'percentage_geb_buiten_nederland_herkmst_buiten_europa',  # Ordinal
    'aantal_part_huishoudens',
    'aantal_eenpersoonshuishoudens',
    'aantal_meerpersoonshuishoudens_zonder_kind',
    'aantal_eenouderhuishoudens',
    'aantal_tweeouderhuishoudens',
    # 'gemiddelde_huishoudensgrootte',          # nth column in category
    'aantal_woningen',
    # 'aantal_woningen_bouwjaar_voor_1945',     # Deemed insignificant
    # 'aantal_woningen_bouwjaar_45_tot_65',     # Deemed insignificant
    # 'aantal_woningen_bouwjaar_65_tot_75',     # Deemed insignificant
    # 'aantal_woningen_bouwjaar_75_tot_85',     # Deemed insignificant
    # 'aantal_woningen_bouwjaar_85_tot_95',     # Deemed insignificant
    # 'aantal_woningen_bouwjaar_95_tot_05',     # Deemed insignificant
    # 'aantal_woningen_bouwjaar_05_tot_15',     # Deemed insignificant
    # 'aantal_woningen_bouwjaar_15_en_later',   # Deemed insignificant
    'aantal_meergezins_woningen',
    # 'percentage_koopwoningen',                # nth column in category
    'percentage_huurwoningen',                # Ordinal, but many categories
    'aantal_huurwoningen_in_bezit_woningcorporaties',
    'aantal_niet_bewoonde_woningen',
    'gemiddelde_woz_waarde_woning',
    'aantal_personen_met_uitkering_onder_aowlft',
    # 'omgevingsadressendichtheid',             # Deemed insignificant
    # 'stedelijkheid',                          # Ordinal
    'BevolkingAantal_inwoners_aantal',
    'BevolkingBurgerlijke_staatGehuwd_aantal',
    'BevolkingBurgerlijke_staatGescheiden_aantal',
    'BevolkingPersonen_met_een_migratieachtergrondNietwestersMarokko_aantal',
    'BevolkingPersonen_met_een_migratieachtergrondNietwestersNederlandse_Antillen_en_Aruba_aantal',
    'BevolkingPersonen_met_een_migratieachtergrondNietwestersSuriname_aantal',
    'BevolkingPersonen_met_een_migratieachtergrondNietwestersTurkije_aantal',
    'BevolkingPersonen_met_een_migratieachtergrondNietwestersOverig_nietwesters_aantal',
    'OpleidingsniveauOpleidingsniveau_laag_aantal',
    'OpleidingsniveauOpleidingsniveau_hoog_aantal',
    'ArbeidNettoarbeidsparticipatie_',
    'InkomenInkomen_van_personenGemiddeld_inkomen_per_inkomensontvanger_x_1_000_euro',
    'InkomenInkomen_van_personenGemiddeld_inkomen_per_inwoner_x_1_000_euro',
    # 'InkomenInkomen_van_personen40_personen_met_laagste_inkomen_',                # Deemed insignificant
    # 'InkomenInkomen_van_personen20_personen_met_hoogste_inkomen_',                # Deemed insignificant
    'InkomenInkomen_van_huishoudensGem_gestandaardiseerd_inkomen_van_huish_x_1_000_euro',
    'InkomenInkomen_van_huishoudens40_huishoudens_met_laagste_inkomen_',
    'InkomenInkomen_van_huishoudens20_huishoudens_met_hoogste_inkomen_',
    'InkomenInkomen_van_huishoudensHuishoudens_met_een_laag_inkomen_',
    'InkomenInkomen_van_huishoudensHuish_onder_of_rond_sociaal_minimum_',
    'InkomenInkomen_van_huishoudensHuishoudens_tot_110_van_sociaal_minimum_',
    'InkomenInkomen_van_huishoudensHuishoudens_tot_120_van_sociaal_minimum_',
    'InkomenInkomen_van_huishoudensMediaan_vermogen_van_particuliere_huish_x_1_000_euro',
    'Sociale_zekerheidPersonen_per_soort_uitkering_Bijstand_aantal',
    'Sociale_zekerheidPersonen_per_soort_uitkering_AO_aantal',
    'Sociale_zekerheidPersonen_per_soort_uitkering_WW_aantal',
    'Sociale_zekerheidPersonen_per_soort_uitkering_AOW_aantal',
    'ZorgJongeren_met_jeugdzorg_in_natura_aantal',
    'ZorgPercentage_jongeren_met_jeugdzorg_',
    'MotorvoertuigenPersonenautosPersonenautos_per_huishouden_per_huishouden',
    'Siblings',
    # 'Geboortejaarmaand_relatie_ouder1',       # We construct age from this
    # 'Geboorteland_relatie_ouder1',            # Categorical
    # 'Geslacht_relatie_ouder1',                # Boolean
    # 'Geboortejaarmaand_relatie_ouder2',       # We construct age from this
    # 'Geboorteland_relatie_ouder2',            # Categorical
    # 'Geslacht_relatie_ouder2',                # Boolean
    # 'Gevaccineerd_ouder1',                    # Boolean
    # 'Gevaccineerd_ouder2',                    # Boolean
    # 'Geboorteland_cat',                       # Categorical
    # 'Geboorteland_relatie_ouder1_cat',        # Categorical
    # 'Geboorteland_relatie_ouder2_cat',        # Categorical
    # 'Geboorteland_cat3',                      # Categorical
    # 'Geboorteland_relatie_ouder1_cat3',       # Categorical
    # 'Geboorteland_relatie_ouder2_cat3',       # Categorical
    # 'CBS_indeling'                            # Categorical
]]

In [8]:
# Convert all columns that are of dtype object to float64, if error proceed with next column
for col in X_schoolgaanden_numericals.columns:
    if X_schoolgaanden_numericals[col].dtype == 'object':
        try:
            X_schoolgaanden_numericals[col] = X_schoolgaanden_numericals[col].str.replace(',', '.').astype('float64')
        except:
            continue

In [9]:
# Transform absolute numbers to relative numbers
X_schoolgaanden_numericals['aandeel_mannen'] = X_schoolgaanden_numericals['aantal_mannen'] / X_schoolgaanden_numericals['aantal_inwoners']
X_schoolgaanden_numericals['aandeel_vrouwen'] = X_schoolgaanden_numericals['aantal_vrouwen'] / X_schoolgaanden_numericals['aantal_inwoners']
X_schoolgaanden_numericals['aandeel_inwoners_0_tot_15_jaar'] = X_schoolgaanden_numericals['aantal_inwoners_0_tot_15_jaar'] / X_schoolgaanden_numericals['aantal_inwoners']
X_schoolgaanden_numericals['aandeel_inwoners_15_tot_25_jaar'] = X_schoolgaanden_numericals['aantal_inwoners_15_tot_25_jaar'] / X_schoolgaanden_numericals['aantal_inwoners']
X_schoolgaanden_numericals['aandeel_inwoners_25_tot_45_jaar'] = X_schoolgaanden_numericals['aantal_inwoners_25_tot_45_jaar'] / X_schoolgaanden_numericals['aantal_inwoners']
X_schoolgaanden_numericals['aandeel_inwoners_45_tot_65_jaar'] = X_schoolgaanden_numericals['aantal_inwoners_45_tot_65_jaar'] / X_schoolgaanden_numericals['aantal_inwoners']
# X_schoolgaanden_numericals['aandeel_inwoners_65_jaar_en_ouder'] = X_schoolgaanden_numericals['aantal_inwoners_65_jaar_en_ouder'] / X_schoolgaanden_numericals['aantal_inwoners']

X_schoolgaanden_numericals['BevolkingBurgerlijke_staatGehuwd_aandeel'] = X_schoolgaanden_numericals['BevolkingBurgerlijke_staatGehuwd_aantal'] / X_schoolgaanden_numericals['BevolkingAantal_inwoners_aantal']
X_schoolgaanden_numericals['BevolkingBurgerlijke_staatGescheiden_aandeel'] = X_schoolgaanden_numericals['BevolkingBurgerlijke_staatGescheiden_aantal'] / X_schoolgaanden_numericals['BevolkingAantal_inwoners_aantal']
X_schoolgaanden_numericals['BevolkingPersonen_met_een_migratieachtergrondNietwestersMarokko_aandeel'] = X_schoolgaanden_numericals['BevolkingPersonen_met_een_migratieachtergrondNietwestersMarokko_aantal'] / X_schoolgaanden_numericals['BevolkingAantal_inwoners_aantal']
X_schoolgaanden_numericals['BevolkingPersonen_met_een_migratieachtergrondNietwestersNederlandse_Antillen_en_Aruba_aandeel'] = X_schoolgaanden_numericals['BevolkingPersonen_met_een_migratieachtergrondNietwestersNederlandse_Antillen_en_Aruba_aantal'] / X_schoolgaanden_numericals['BevolkingAantal_inwoners_aantal']
X_schoolgaanden_numericals['BevolkingPersonen_met_een_migratieachtergrondNietwestersSuriname_aandeel'] = X_schoolgaanden_numericals['BevolkingPersonen_met_een_migratieachtergrondNietwestersSuriname_aantal'] / X_schoolgaanden_numericals['BevolkingAantal_inwoners_aantal']
X_schoolgaanden_numericals['BevolkingPersonen_met_een_migratieachtergrondNietwestersTurkije_aandeel'] = X_schoolgaanden_numericals['BevolkingPersonen_met_een_migratieachtergrondNietwestersTurkije_aantal'] / X_schoolgaanden_numericals['BevolkingAantal_inwoners_aantal']
X_schoolgaanden_numericals['BevolkingPersonen_met_een_migratieachtergrondNietwestersOverig_nietwesters_aandeel'] = X_schoolgaanden_numericals['BevolkingPersonen_met_een_migratieachtergrondNietwestersOverig_nietwesters_aantal'] / X_schoolgaanden_numericals['BevolkingAantal_inwoners_aantal']
X_schoolgaanden_numericals['OpleidingsniveauOpleidingsniveau_laag_aandeel'] = X_schoolgaanden_numericals['OpleidingsniveauOpleidingsniveau_laag_aantal'] / X_schoolgaanden_numericals['BevolkingAantal_inwoners_aantal']
X_schoolgaanden_numericals['OpleidingsniveauOpleidingsniveau_hoog_aandeel'] = X_schoolgaanden_numericals['OpleidingsniveauOpleidingsniveau_hoog_aantal'] / X_schoolgaanden_numericals['BevolkingAantal_inwoners_aantal']
X_schoolgaanden_numericals['Sociale_zekerheidPersonen_per_soort_uitkering_Bijstand_aandeel'] = X_schoolgaanden_numericals['Sociale_zekerheidPersonen_per_soort_uitkering_Bijstand_aantal'] / X_schoolgaanden_numericals['BevolkingAantal_inwoners_aantal']
X_schoolgaanden_numericals['Sociale_zekerheidPersonen_per_soort_uitkering_AO_aandeel'] = X_schoolgaanden_numericals['Sociale_zekerheidPersonen_per_soort_uitkering_AO_aantal'] / X_schoolgaanden_numericals['BevolkingAantal_inwoners_aantal']
X_schoolgaanden_numericals['Sociale_zekerheidPersonen_per_soort_uitkering_WW_aandeel'] = X_schoolgaanden_numericals['Sociale_zekerheidPersonen_per_soort_uitkering_WW_aantal'] / X_schoolgaanden_numericals['BevolkingAantal_inwoners_aantal']
X_schoolgaanden_numericals['Sociale_zekerheidPersonen_per_soort_uitkering_AOW_aandeel'] = X_schoolgaanden_numericals['Sociale_zekerheidPersonen_per_soort_uitkering_AOW_aantal'] / X_schoolgaanden_numericals['BevolkingAantal_inwoners_aantal']
X_schoolgaanden_numericals['ZorgJongeren_met_jeugdzorg_in_natura_aandeel'] = X_schoolgaanden_numericals['ZorgJongeren_met_jeugdzorg_in_natura_aantal'] / X_schoolgaanden_numericals['BevolkingAantal_inwoners_aantal']

X_schoolgaanden_numericals['aandeel_eenpersoonshuishoudens'] = X_schoolgaanden_numericals['aantal_eenpersoonshuishoudens'] / X_schoolgaanden_numericals['aantal_part_huishoudens']
X_schoolgaanden_numericals['aandeel_meerpersoonshuishoudens_zonder_kind'] = X_schoolgaanden_numericals['aantal_meerpersoonshuishoudens_zonder_kind'] / X_schoolgaanden_numericals['aantal_part_huishoudens']
X_schoolgaanden_numericals['aandeel_eenouderhuishoudens'] = X_schoolgaanden_numericals['aantal_eenouderhuishoudens'] / X_schoolgaanden_numericals['aantal_part_huishoudens']
X_schoolgaanden_numericals['aandeel_tweeouderhuishoudens'] = X_schoolgaanden_numericals['aantal_tweeouderhuishoudens'] / X_schoolgaanden_numericals['aantal_part_huishoudens']

X_schoolgaanden_numericals['aandeel_personen_met_uitkering_onder_aowlft'] = X_schoolgaanden_numericals['aantal_personen_met_uitkering_onder_aowlft'] / X_schoolgaanden_numericals['aantal_inwoners']

X_schoolgaanden_numericals['aandeel_meergezins_woningen'] = X_schoolgaanden_numericals['aantal_meergezins_woningen'] / X_schoolgaanden_numericals['aantal_woningen']
X_schoolgaanden_numericals['aandeel_huurwoningen_in_bezit_woningcorporaties'] = X_schoolgaanden_numericals['aantal_huurwoningen_in_bezit_woningcorporaties'] / X_schoolgaanden_numericals['aantal_woningen']
X_schoolgaanden_numericals['aandeel_niet_bewoonde_woningen'] = X_schoolgaanden_numericals['aantal_niet_bewoonde_woningen'] / X_schoolgaanden_numericals['aantal_woningen']

# Drop all columns that are not needed anymore
X_schoolgaanden_numericals = X_schoolgaanden_numericals.drop(columns=['aantal_mannen',
                                            'aantal_vrouwen',
                                            'aantal_inwoners_0_tot_15_jaar',
                                            'aantal_inwoners_15_tot_25_jaar',
                                            'aantal_inwoners_25_tot_45_jaar',
                                            'aantal_inwoners_45_tot_65_jaar',
                                            # 'aantal_inwoners_65_jaar_en_ouder',
                                            'BevolkingAantal_inwoners_aantal',
                                            'BevolkingBurgerlijke_staatGehuwd_aantal',
                                            'BevolkingBurgerlijke_staatGescheiden_aantal',
                                            'BevolkingPersonen_met_een_migratieachtergrondNietwestersMarokko_aantal',
                                            'BevolkingPersonen_met_een_migratieachtergrondNietwestersNederlandse_Antillen_en_Aruba_aantal',
                                            'BevolkingPersonen_met_een_migratieachtergrondNietwestersSuriname_aantal',
                                            'BevolkingPersonen_met_een_migratieachtergrondNietwestersTurkije_aantal',
                                            'BevolkingPersonen_met_een_migratieachtergrondNietwestersOverig_nietwesters_aantal',
                                            'OpleidingsniveauOpleidingsniveau_laag_aantal',
                                            'OpleidingsniveauOpleidingsniveau_hoog_aantal',
                                            'Sociale_zekerheidPersonen_per_soort_uitkering_Bijstand_aantal',
                                            'Sociale_zekerheidPersonen_per_soort_uitkering_AO_aantal',
                                            'Sociale_zekerheidPersonen_per_soort_uitkering_WW_aantal',
                                            'Sociale_zekerheidPersonen_per_soort_uitkering_AOW_aantal',
                                            'ZorgJongeren_met_jeugdzorg_in_natura_aantal',
                                            'aantal_inwoners',
                                            'aantal_eenpersoonshuishoudens',
                                            'aantal_meerpersoonshuishoudens_zonder_kind',
                                            'aantal_eenouderhuishoudens',
                                            'aantal_tweeouderhuishoudens',
                                            'aantal_part_huishoudens',
                                            'aantal_personen_met_uitkering_onder_aowlft',
                                            'aantal_meergezins_woningen',
                                            'aantal_huurwoningen_in_bezit_woningcorporaties',
                                            'aantal_niet_bewoonde_woningen',
                                            'aantal_woningen'])

##### 3.1.2 Age features

In [10]:
# Assuming schoolgaanden is your DataFrame
schoolgaanden['Geboortejaarmaand_relatie_ouder1'] = pd.to_datetime(schoolgaanden['Geboortejaarmaand_relatie_ouder1']).dt.to_period('M')
schoolgaanden['Geboortejaarmaand_relatie_ouder2'] = pd.to_datetime(schoolgaanden['Geboortejaarmaand_relatie_ouder2']).dt.to_period('M')

# Convert NaT to NaN
schoolgaanden['Geboortejaarmaand'] = schoolgaanden['Geboortejaarmaand'].fillna(pd.NaT)
schoolgaanden['Geboortejaarmaand_relatie_ouder1'] = schoolgaanden['Geboortejaarmaand_relatie_ouder1'].fillna(pd.NaT)
schoolgaanden['Geboortejaarmaand_relatie_ouder2'] = schoolgaanden['Geboortejaarmaand_relatie_ouder2'].fillna(pd.NaT)

# Construct age features
X_schoolgaanden_numericals['Leeftijd_moeder'] = (schoolgaanden['Geboortejaarmaand'] - schoolgaanden['Geboortejaarmaand_relatie_ouder1']).apply(lambda x: x.n if pd.notnull(x) else x)/12
X_schoolgaanden_numericals['Leeftijd_ouder2'] = (schoolgaanden['Geboortejaarmaand'] - schoolgaanden['Geboortejaarmaand_relatie_ouder2']).apply(lambda x: x.n if pd.notnull(x) else x)/12

# Convert NaT to NaN
X_schoolgaanden_numericals['Leeftijd_moeder'] = X_schoolgaanden_numericals['Leeftijd_moeder'].fillna(np.nan)
X_schoolgaanden_numericals['Leeftijd_ouder2'] = X_schoolgaanden_numericals['Leeftijd_ouder2'].fillna(np.nan)

In [ ]:
# Plot the distribtuion of X_schoolgaanden_numericals['Leeftijd_moeder'] and X_schoolgaanden_numericals['Leeftijd_ouder2'], excluding NaN values
plt.hist(X_schoolgaanden_numericals['Leeftijd_moeder'].dropna(), bins=50, alpha=0.5, label='Leeftijd_moeder')
plt.hist(X_schoolgaanden_numericals['Leeftijd_ouder2'].dropna(), bins=50, alpha=0.5, label='Leeftijd_ouder2')

# Print the min and max values of the age features
print(f"Min leeftijd moeder: {round(X_schoolgaanden_numericals['Leeftijd_moeder'].min(),2)}, Max leeftijd moeder: {round(X_schoolgaanden_numericals['Leeftijd_moeder'].max(),2)}")
print(f"Min leeftijd Ouder2: {round(X_schoolgaanden_numericals['Leeftijd_ouder2'].min(),2)}, Max leeftijd Ouder2: {round(X_schoolgaanden_numericals['Leeftijd_ouder2'].max(),2)}")

#### 3.2 Categorical features

In [12]:
# Get categorical columns
X_schoolgaanden_categoricals = schoolgaanden[[
    # 'Geboorteland',                       # Lies in CBS_indeling, too sparse
    # 'Geboorteland_relatie_ouder1',        # Lies in CBS_indeling, too sparse
    # 'Geboorteland_relatie_ouder2',        # Lies in CBS_indeling, too sparse
    # 'Geboorteland_cat',                   # Lies in CBS_indeling, too sparse
    # 'Geboorteland_relatie_ouder1_cat',    # Lies in CBS_indeling, too sparse
    # 'Geboorteland_relatie_ouder2_cat',    # Lies in CBS_indeling, too sparse
    # 'Geboorteland_cat3',                  # Lies in CBS_indeling, too sparse
    # 'Geboorteland_relatie_ouder1_cat3',   # Lies in CBS_indeling, too sparse
    # 'Geboorteland_relatie_ouder2_cat3',   # Lies in CBS_indeling, too sparse
    # 'percentage_geb_nederland_herkomst_nederland',                # Ordinal, but too many categories
    'percentage_geb_nederland_herkomst_overig_europa',            # Ordinal
    'percentage_geb_nederland_herkomst_buiten_europa',            # Ordinal
    'percentage_geb_buiten_nederland_herkomst_europa',            # Ordinal
    'percentage_geb_buiten_nederland_herkmst_buiten_europa',      # Ordinal
    # 'percentage_huurwoningen',                                    # Ordinal, but too many categories
    'stedelijkheid',                                              # Ordinal
    'CBS_indeling',
    'Toestemming'
    ]]

In [13]:
# Create dummy variables
X_schoolgaanden_categoricals = pd.get_dummies(X_schoolgaanden_categoricals, columns=X_schoolgaanden_categoricals.columns, dummy_na=True, drop_first=False)

In [ ]:
X_schoolgaanden_categoricals

#### 3.3 Boolean features

In [15]:
# Get boolean columns
X_schoolgaanden_booleans = schoolgaanden[[
    'Tweeling',
    'Geslacht',
    'Geslacht_relatie_ouder1',
    'Geslacht_relatie_ouder2',
    'Gevaccineerd_ouder1',
    'Gevaccineerd_ouder2'
    ]]

In [ ]:
X_schoolgaanden_booleans

In [ ]:
# Specify boolean mapping
def boolean_mapper(df):
    bool_mapping = {'Ja': True, 'Nee': False,
                    'Vaccinated': True, 'Not Vaccinated': False,
                    'Mannelijk': True, 'Vrouwelijk': False,
                    'mannelijk': True, 'vrouwelijk': False,
                    1: True, 0: False}
    def convert_to_bool(column):
        return bool_mapping.get(column, np.NaN)

    for col in df.columns:
        df[col] = df[col].apply(convert_to_bool)
    return df

# Apply boolean mapping
boolean_mapper(X_schoolgaanden_booleans)

In [18]:
# Change column names
X_schoolgaanden_booleans.columns = X_schoolgaanden_booleans.columns.str.replace('Geslacht', 'Geslacht=mannelijk')

In [19]:
# Save all one-hot coded column names
one_hot_columns = list(X_schoolgaanden_categoricals.columns) + list(X_schoolgaanden_booleans.columns)

#### 3.4 Target feature

In [20]:
# Store the target variable
y_schoolgaanden = schoolgaanden['Schoolgaanden_status']

In [21]:
# Specify mappings
target_mapping = {'Not Vaccinated': 0,
                  'Vaccinated': 1}

# Apply the mapping to the Series
y_schoolgaanden = y_schoolgaanden.map(target_mapping)

In [ ]:
# Count unique values of y_schoolgaanden
np.unique(y_schoolgaanden, return_counts=True)

### 4. Feature selection

In [ ]:
# Print the percentage of missing values per column
print(f'The percentage of missing values per column is:')
for col in X_schoolgaanden_numericals.columns:
    print(f'{col:<100}: {round(X_schoolgaanden_numericals[col].isnull().mean(), 2)}')

In [ ]:
# Print the percentage of missing values per column
print(f'The percentage of missing values per column is:')
for col in X_schoolgaanden_categoricals.columns:
    print(f'{col:<100}: {round(X_schoolgaanden_categoricals[col].isnull().mean(), 2)}')

In [ ]:
# Print the percentage of missing values per column
print(f'The percentage of missing values per column is:')
for col in X_schoolgaanden_booleans.columns:
    print(f'{col:<100}: {round(X_schoolgaanden_booleans[col].isnull().mean(), 2)}')

### 5. Merge and drop features

In [26]:
# Merge all dataframes
X_schoolgaanden = pd.concat([X_schoolgaanden_numericals, X_schoolgaanden_categoricals, X_schoolgaanden_booleans], axis=1)

# Drop sex columns from booleans, as they have large amounts of missing values and were mostly important for their vaccination statuses
X_schoolgaanden = X_schoolgaanden.drop(columns=['Geslacht=mannelijk_relatie_ouder1', 'Geslacht=mannelijk_relatie_ouder2'])

### 6. Split subsets

In [ ]:
# Split subset of X_schoolgaanden_numericals
X_schoolgaanden_notvaccinated = X_schoolgaanden[y_schoolgaanden == 0]

# Make copy of X_schoolgaanden
X_schoolgaanden_reduced = X_schoolgaanden.copy()

# Delete columns from X_schoolgaanden that have >50% missings in X_schoolgaanden_notvaccinated
for col in X_schoolgaanden.columns:
    if col in X_schoolgaanden_notvaccinated.columns:
        if X_schoolgaanden_notvaccinated[col].isnull().mean() > 0.5:
            X_schoolgaanden_reduced = X_schoolgaanden_reduced.drop(columns=col)

# Make list of strings with column names in X_schoolgaanden containing 'nan'
nan_columns = [col for col in X_schoolgaanden.columns if 'nan' in col]

# Loop over nan_columns in X_schoolgaanden_reduced and calculate if >50% is 'True'
for col in nan_columns:
    col_variable = col.split('_nan')[0]
    if col_variable != 'Toestemming':
        if (1 - X_schoolgaanden_notvaccinated[col].value_counts(normalize=True)[False]) > 0.5:
            X_schoolgaanden_reduced = X_schoolgaanden_reduced.drop(columns=[col for col in X_schoolgaanden_reduced.columns if col_variable in col])

# Show which columns got dropped
print(f'The columns that were dropped from X_schoolgaanden are:')
for col in X_schoolgaanden.columns:
    if col not in X_schoolgaanden_reduced.columns:
        print(f'    {col}')

In [28]:
# Randomly sample 10% of the data
if TEST_instances:
    X_schoolgaanden_reduced = X_schoolgaanden_reduced.sample(frac=0.03, random_state=42)
    y_schoolgaanden = y_schoolgaanden.loc[X_schoolgaanden_reduced.index]

### 7. Missing value imputation

In [29]:
# Initialize the IterativeImputer
if TEST_iterations:
    imputer = IterativeImputer(random_state=42, estimator=RandomForestRegressor(random_state=42), max_iter=3)
else:
    imputer = IterativeImputer(random_state=42, estimator=RandomForestRegressor(random_state=42), max_iter=5)

In [ ]:
# Print the percentages of missing values per column
print(f'The percentage of missing values per column is:')
for col in X_schoolgaanden_reduced.columns:
    print(f'{col:<100}: {round(X_schoolgaanden_reduced[col].isnull().mean(), 2)}')

In [ ]:
# Fit the imputer on the data
imputer.fit(X_schoolgaanden_reduced)

# Transform the data
X_schoolgaanden_imputed = pd.DataFrame(imputer.transform(X_schoolgaanden_reduced), columns=X_schoolgaanden_reduced.columns)

In [ ]:
# Get the top 10 missing value columns
missing_values = X_schoolgaanden_reduced.isnull().sum().sort_values(ascending=False).head(10).index

# Create a 5x2 subplot frame
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(15, 10))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Define custom colors
color_before = 'blue'
color_after = 'salmon'

# Define a formatter function to round to two decimals
formatter = FuncFormatter(lambda x, _: f'{x:.2f}')

# Plot the histograms and bar charts
for i, column in enumerate(missing_values):
    if X_schoolgaanden_reduced[column].dtype == 'float64':
        X_schoolgaanden_reduced[column].plot.hist(ax=axes[i], bins=30, alpha=0.7, label='Voor imputatie', color=color_before)
        X_schoolgaanden_imputed[column].plot.hist(ax=axes[i], bins=30, alpha=0.7, label='Na imputatie', color=color_after)
    else:
        X_schoolgaanden_reduced[column].value_counts().plot.bar(ax=axes[i], alpha=0.7, label='Voor imputatie', color=color_before)
        X_schoolgaanden_imputed[column].value_counts().plot.bar(ax=axes[i], alpha=0.7, label='Na imputatie', color=color_after)
    
    axes[i].set_xlabel(column)
    axes[i].set_ylabel('Frequentie')
    axes[i].xaxis.set_major_formatter(formatter)  # Apply the formatter to the x-axis

# Collect handles and labels from the first subplot
handles, labels = axes[0].get_legend_handles_labels()

# Create a single legend for the entire figure
fig.legend(handles, labels, loc='upper center', ncol=2)

# Adjust layout
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust rect to make space for the legend
plt.show()

### 8. Drop features related to target

In [149]:
# List of columns to drop
columns_to_drop = ['BMR', 'DTP']

# Filter out columns that are not in the DataFrame
columns_to_drop = [col for col in columns_to_drop if col in X_schoolgaanden_imputed.columns]

# Drop the specified columns from the DataFrames
X_schoolgaanden = X_schoolgaanden_imputed.drop(columns=columns_to_drop)

### 9. Round boolean columns to 0 or 1

In [150]:
# Round values within 'bools' columns to 0 or 1 if they exist
for col in one_hot_columns:
    if col in X_schoolgaanden.columns:
        X_schoolgaanden[col] = X_schoolgaanden[col].round().astype(bool)

### 10. Save .pkl's

In [151]:
# Save original features
X_schoolgaanden.to_pickle('4_Data/Pickles/X_schoolgaanden.pkl')

# Save subs
X_schoolgaanden_notvaccinated = X_schoolgaanden.loc[y_schoolgaanden == 0]
X_schoolgaanden_vaccinated = X_schoolgaanden.loc[y_schoolgaanden == 1]

# Save
X_schoolgaanden_notvaccinated.to_pickle('4_Data/Pickles/X_schoolgaanden_notvaccinated.pkl')
X_schoolgaanden_vaccinated.to_pickle('4_Data/Pickles/X_schoolgaanden_vaccinated.pkl')

In [152]:
# Save scaler for numerical subsets
scaler = RobustScaler()

# Save y_schoolgaanden
np.save('4_Data\Arrays\y_schoolgaanden.npy', y_schoolgaanden)

# Save columns of dtype float of X_schoolgaanden
X_schoolgaanden_notvaccinated_numericals = X_schoolgaanden_notvaccinated.select_dtypes(include='float64')
X_schoolgaanden_notvaccinated_numericals = pd.DataFrame(scaler.fit_transform(X_schoolgaanden_notvaccinated_numericals), columns=X_schoolgaanden_notvaccinated_numericals.columns)
X_schoolgaanden_notvaccinated_numericals.to_pickle('4_Data/Pickles/X_schoolgaanden_notvaccinated_numericals.pkl')

# Save columns of dtype bool of X_schoolgaanden
X_schoolgaanden_notvaccinated_booleans = X_schoolgaanden_notvaccinated.select_dtypes(exclude='float64')
X_schoolgaanden_notvaccinated_booleans.to_pickle('4_Data/Pickles/X_schoolgaanden_notvaccinated_booleans.pkl')